### Predicting formation enthalpies for solid solutions of Lanthanides Orthophosphates


In [1]:
import os
import sys
import inspect

print(sys.version)

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
grandparentdir = os.path.dirname(parentdir)

sys.path.insert(0, grandparentdir) 

3.9.6 (default, Nov 10 2023, 13:38:27) 
[Clang 15.0.0 (clang-1500.1.0.2.5)]


In [2]:
import read_data
import featureSpan
import lasso
import Utils
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import scipy.optimize
import pandas as pd
import seaborn as sns
import scipy.stats as ss
from sklearn.linear_model import Lasso
from itertools import combinations, product
import itertools
import math
import pandas as pd

## Monazite

#### Load data from files

In [3]:
list1 = read_data.readData("../../data/DATA_HE_monazite.dat")
print("Shape of List1 is {}".format(list1.shape))
print(list1)

Shape of List1 is (525, 4)
[[5.70000000e+01 5.80000000e+01 7.50000000e-01 1.29717527e-01]
 [5.70000000e+01 5.80000000e+01 6.25000000e-01 1.90974365e-01]
 [5.70000000e+01 5.80000000e+01 5.00000000e-01 1.92712564e-01]
 ...
 [7.00000000e+01 7.10000000e+01 5.00000000e-01 2.12444086e-02]
 [7.00000000e+01 7.10000000e+01 3.75000000e-01 2.22639048e-02]
 [7.00000000e+01 7.10000000e+01 2.50000000e-01 2.09098752e-02]]


In [4]:
list2 = read_data.readCSVData_simplified("../../data/Data_Ln-monazite.csv")
print("Shape of List2 is {}".format(list2.shape))
print((list2))

Shape of List2 is (15, 2)
[[139.1         77.61001595]
 [146.5         75.78168414]
 [150.1         74.20076494]
 [154.1         72.98174154]
 [157.4         71.90916336]
 [160.3         70.88675537]
 [163.1         70.00137871]
 [165.2         69.15236746]
 [164.5         68.41325589]
 [165.4         67.64549329]
 [167.3         67.05867559]
 [169.1         66.33683909]
 [170.8         65.60276254]
 [172.2         65.17192527]
 [171.          64.45110847]]


#### Generated Elemental Training/Testing sets from loaded data

In [5]:
listX, listY = featureSpan.generateFeatures(list1, list2)
X = np.asarray(listX)
Y = np.asarray(listY)
print(X.shape)
print(Y.shape)

elemental_list = ["m", "1/m", "m^2", "(1/m)^2", "(1-m)", "1/(1-m)", "(1-m)^2", "(1/(1-m))^2", "[AD:Young]", "[AD:Volume]", "1/[AD:Young]", "1/[AD:Volume]", "[AD:Young]^2", "[AD:Volume]^2", "(1/[AD:Young])^2", "(1/[AD:Volume])^2", "[AM:Young]", "[AM:Volume]", "1/[AM:Young]", "1/[AM:Volume]","[AM:Young]^2", "[AM:Volume]^2", "(1/[AM:Young])^2", "(1/[AM:Volume]])^2"]
print(len(elemental_list))
print(elemental_list)
m, n = X.shape

elemental_features = []

for i in elemental_list:
    elemental_features.append("("+i+")")
        
print("The elemental features are: \n {}".format(elemental_features))

(525, 24)
(525,)
24
['m', '1/m', 'm^2', '(1/m)^2', '(1-m)', '1/(1-m)', '(1-m)^2', '(1/(1-m))^2', '[AD:Young]', '[AD:Volume]', '1/[AD:Young]', '1/[AD:Volume]', '[AD:Young]^2', '[AD:Volume]^2', '(1/[AD:Young])^2', '(1/[AD:Volume])^2', '[AM:Young]', '[AM:Volume]', '1/[AM:Young]', '1/[AM:Volume]', '[AM:Young]^2', '[AM:Volume]^2', '(1/[AM:Young])^2', '(1/[AM:Volume]])^2']
The elemental features are: 
 ['(m)', '(1/m)', '(m^2)', '((1/m)^2)', '((1-m))', '(1/(1-m))', '((1-m)^2)', '((1/(1-m))^2)', '([AD:Young])', '([AD:Volume])', '(1/[AD:Young])', '(1/[AD:Volume])', '([AD:Young]^2)', '([AD:Volume]^2)', '((1/[AD:Young])^2)', '((1/[AD:Volume])^2)', '([AM:Young])', '([AM:Volume])', '(1/[AM:Young])', '(1/[AM:Volume])', '([AM:Young]^2)', '([AM:Volume]^2)', '((1/[AM:Young])^2)', '((1/[AM:Volume]])^2)']


In [6]:
dfX = pd.DataFrame(data=X, columns=elemental_features)

new_features=[]
new_columns = []

for i in range(n):
    for j in range(i):
        new_features.append(elemental_features[i]+"*"+elemental_features[j])
        new_columns.append(dfX[elemental_features[i]].values * dfX[elemental_features[j]].values)


for i in range(n):
    for j in range(i):
        for k in range(j):
            new_features.append(elemental_features[i]+"*"+elemental_features[j]+"*"+elemental_features[k])
            new_columns.append(dfX[elemental_features[i]] * dfX[elemental_features[j]] * dfX[elemental_features[k]])
            

#new_columns = np.reshape(np.asarray(new_columns))
            
new_columns = np.asarray(new_columns)
dfX = pd.concat(
    [
        dfX,
        pd.DataFrame(
            new_columns.T, 
            index=dfX.index, 
            columns=new_features
        )
    ], axis=1
)

dfX.head()

dfX.std() == 0
dfX=dfX.loc[:, dfX.std() > 0]
dfX.head()

,(m),(1/m),(m^2),((1/m)^2),((1-m)),(1/(1-m)),((1-m)^2),((1/(1-m))^2),([AD:Young]),([AD:Volume]),...,((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AD:Young]^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AD:Volume]^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*((1/[AD:Young])^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*((1/[AD:Volume])^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Young]),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Volume]),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*(1/[AM:Young]),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*(1/[AM:Volume]),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Young]^2),((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Volume]^2)
0,0.750,1.333333,0.562500,1.777778,0.250,4.000000,0.062500,16.000000,3.7,0.914166,...,1.141309e-07,6.967065e-09,6.089707e-10,9.975848e-09,0.000001,6.393987e-07,5.838102e-11,1.086996e-10,0.00017,0.000049
1,0.625,1.600000,0.390625,2.560000,0.375,2.666667,0.140625,7.111111,3.7,0.914166,...,1.141309e-07,6.967065e-09,6.089707e-10,9.975848e-09,0.000001,6.393987e-07,5.838102e-11,1.086996e-10,0.00017,0.000049
2,0.500,2.000000,0.250000,4.000000,0.500,2.000000,0.250000,4.000000,3.7,0.914166,...,1.141309e-07,6.967065e-09,6.089707e-10,9.975848e-09,0.000001,6.393987e-07,5.838102e-11,1.086996e-10,0.00017,0.000049
3,0.375,2.666667,0.140625,7.111111,0.625,1.600000,0.390625,2.560000,3.7,0.914166,...,1.141309e-07,6.967065e-09,6.089707e-10,9.975848e-09,0.000001,6.393987e-07,5.838102e-11,1.086996e-10,0.00017,0.000049
4,0.250,4.000000,0.062500,16.000000,0.750,1.333333,0.562500,1.777778,3.7,0.914166,...,1.141309e-07,6.967065e-09,6.089707e-10,9.975848e-09,0.000001,6.393987e-07,5.838102e-11,1.086996e-10,0.00017,0.000049


In [7]:
print(len(dfX.columns.values))
np.array(dfX.columns.values)

2320


array(['(m)', '(1/m)', '(m^2)', ...,
       '((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*(1/[AM:Volume])',
       '((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Young]^2)',
       '((1/[AM:Volume]])^2)*((1/[AM:Young])^2)*([AM:Volume]^2)'],
      dtype=object)

fitting of Lasso with a given $\lambda$

#### Analyze the behavior of Lasso with the decrease of $\lambda$

In [10]:
#### Define a function which fits Lasso to have no more nonzero coefficients than a given threshold 
def LassoSelect(X, Y, min, max, step, threshold, standardization = True):
    
    scaler = StandardScaler()
    scaler.fit(X)
    X_standardized = scaler.transform(X)
        
    found = False
    for lmbda in np.arange (min, max, step):
        coef, selected_indices, selected_features, MAE, MSE, ME = LassoFit(lmbda, X.copy(), Y.copy())
        if len(selected_indices) <= threshold:
            found = True
            break
    
    if found:
        print("FOUND with threshold: {}".format(threshold))
        print("Lambda: {}, nnz: {}, MAE: {}, MSE: {}, MAPE: {}".format(lmbda, len(selected_indices), MAE, MSE, ME))
            
    else:
        print("NOT FOUND with threshold: {}".format(threshold))
        print("Closest are: ")
        print("Lambda: {}, nnz: {}, MAE: {}, MSE: {}, MAPE: {}".format(lmbda, len(selected_indices), MAE, MSE, ME))
     
    X_reduced = X[selected_features]
    
    return X_reduced


X_reduced = LassoSelect(dfX, Y, 0.001, 0.101, 0.005, 30)
X_reduced.head()

FOUND with threshold: 30
Lambda: 0.006, nnz: 13, MAE: 0.04966075829021671, MSE: 0.006238940879980003, MAPE: 0.16509314785030518


,([AD:Young])*((1/m)^2)*(1/m),([AD:Young])*((1-m))*(m),([AD:Volume])*((1-m))*(m),([AD:Young]^2)*((1/m)^2)*(1/m),([AD:Young]^2)*((1-m)^2)*(m),([AD:Volume]^2)*((1-m))*(m),([AD:Volume]^2)*((1-m)^2)*(m),([AD:Volume]^2)*((1-m)^2)*(m^2),([AD:Volume]^2)*(1/[AD:Young])*([AD:Volume]),([AM:Young]^2)*((1-m)^2)*(m^2),([AM:Young]^2)*([AD:Volume])*(m),([AM:Young]^2)*([AD:Volume])*(1/(1-m)),([AM:Young]^2)*([AM:Young])*([AD:Volume]^2)
0,8.770370,0.693750,0.171406,32.450370,0.641719,0.156694,0.039173,0.029380,0.206478,716.900625,13981.143625,74566.099335,2.433519e+06
1,15.155200,0.867188,0.214258,56.074240,1.203223,0.195867,0.073450,0.045906,0.206478,1120.157227,11650.953021,49710.732890,2.433519e+06
2,29.600000,0.925000,0.228541,109.520000,1.711250,0.208925,0.104462,0.052231,0.206478,1274.490000,9320.762417,37283.049668,2.433519e+06
3,70.162963,0.867188,0.214258,259.602963,2.005371,0.195867,0.122417,0.045906,0.206478,1120.157227,6990.571813,29826.439734,2.433519e+06
4,236.800000,0.693750,0.171406,876.160000,1.925156,0.156694,0.117520,0.029380,0.206478,716.900625,4660.381208,24855.366445,2.433519e+06


#### Lasso with $\ell_0$

In [11]:
def LassoL0(X, Y, nnz):    
    nr, nc = X.shape
    X = np.column_stack((X, np.ones(nr)))
    se_min = np.inner(Y, Y)
    coef_min, permu_min = None, None
    for permu in combinations(range(nc), nnz):
        X_ls = X[:, permu + (-1,)]
        coef, se, __1, __2 = np.linalg.lstsq(X_ls, Y, rcond=-1)
        try:
            if se[0] < se_min: 
                se_min = se[0]
                coef_min, permu_min = coef, permu
        except:
            pass
        
    return coef_min, permu_min

In [12]:
def LassoL0Fit(X_reduced, Y, nnz, log=True):
    
    scaler = StandardScaler()
    scaler.fit(X_reduced)
    X_std = scaler.transform(X_reduced)
    
    nr, nc = X_reduced.shape
   
    coefficients, selected_indices = LassoL0(X_std, Y, nnz)
   
    coefficients = np.array(coefficients)
    selected_indices = np.array(selected_indices)
    feature_reduced = np.array(X_reduced.columns.values)
    feature_list_selected = feature_reduced[selected_indices]
    
    X_selected = X_reduced[feature_list_selected]

    mean_selected = X_selected.mean()
    std_selected = X_selected.std()

    
    if log:
        print("Lasso: selected coefficients are: {}".format(coefficients))
        print("Lasso: selected features are: {}".format(feature_list_selected))
        
    #-mean/std
    mean_std = []
    for i in range(len(selected_indices)):
        mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
 
    sum_mean_std = sum(mean_std)

    for i in range(len(selected_indices)):
        coefficients[i] = coefficients[i] / std_selected[i]

    
    coefficients[len(selected_indices)] -= sum_mean_std
    
    function = str((coefficients[0]))+" * "+feature_list_selected[0]
    
    for i in range(1, len(selected_indices)):
        if coefficients[i] >= 0:
            function += " + " + str(coefficients[i])+" * "+feature_list_selected[i]
        else:
            function += " - " + str(abs(coefficients[i]))+" * "+feature_list_selected[i]

    
    if coefficients[len(selected_indices)] >= 0:
        function += " + " + str(coefficients[len(selected_indices)])
    else:
        function += " - " + str(abs(coefficients[len(selected_indices)]))
    
    if log:
        print("Constructed function is: {}".format(function))

    X_selected = np.column_stack((X_selected, np.ones(X_selected.shape[0])))
    Y_predict = X_selected[:,0] * coefficients[0]

    for i in range(1,len(selected_indices)+1):
        Y_predict = Y_predict + X_selected[:,i] * coefficients[i]
    
    if log:
        Utils.print_error(Y.copy(),Y_predict,"Lasso L0: {} coef".format(nnz))
    
    return Y_predict, coefficients, selected_indices

In [13]:
LassoL0Fit(X_reduced, Y.copy(), 1);

Lasso: selected coefficients are: [1.9567842  1.65053546]
Lasso: selected features are: ['([AD:Volume]^2)*((1-m))*(m)']
Constructed function is: 0.9246943428407507 * ([AD:Volume]^2)*((1-m))*(m) + 0.017339285855854758
Lasso L0: 1 coef
Mean absolute error: 0.062277848479921454
Mean squared error: 0.009550658795500384
Mean absolute percentage error: 0.19518882815326008


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18163/603291486.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18163/603291486.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [14]:
LassoL0Fit(X_reduced, Y.copy(), 2);

Lasso: selected coefficients are: [0.12644657 1.83559451 1.65053546]
Lasso: selected features are: ['([AD:Volume])*((1-m))*(m)' '([AD:Volume]^2)*((1-m))*(m)']
Constructed function is: 0.35728718794387804 * ([AD:Volume])*((1-m))*(m) + 0.8674251647103202 * ([AD:Volume]^2)*((1-m))*(m) - 0.06608239821022543
Lasso L0: 2 coef
Mean absolute error: 0.05851171999934852
Mean squared error: 0.00824886689717345
Mean absolute percentage error: 0.2506786655487066


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18163/603291486.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18163/603291486.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [15]:
LassoL0Fit(X_reduced, Y.copy(), 3);

Lasso: selected coefficients are: [-0.08925325  1.83701654  0.17485762  1.65053546]
Lasso: selected features are: ['([AD:Young]^2)*((1/m)^2)*(1/m)' '([AD:Volume]^2)*((1-m))*(m)'
 '([AD:Volume]^2)*((1-m)^2)*(m)']
Constructed function is: -4.0712995886053066e-05 * ([AD:Young]^2)*((1/m)^2)*(1/m) + 0.8680971549857976 * ([AD:Volume]^2)*((1-m))*(m) + 0.15271628542669957 * ([AD:Volume]^2)*((1-m)^2)*(m) + 0.020689739933246853
Lasso L0: 3 coef
Mean absolute error: 0.05461822516282557
Mean squared error: 0.00685720147844757
Mean absolute percentage error: 0.21480063446992895


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18163/603291486.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18163/603291486.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [16]:
LassoL0Fit(X_reduced, Y.copy(), 4);

Lasso: selected coefficients are: [ 0.09537768 -0.09231717  1.74430229  0.18022737  1.65053546]
Lasso: selected features are: ['([AD:Young])*((1-m))*(m)' '([AD:Young]^2)*((1/m)^2)*(1/m)'
 '([AD:Volume]^2)*((1-m))*(m)' '([AD:Volume]^2)*((1-m)^2)*(m)']
Constructed function is: 0.10281312939968847 * ([AD:Young])*((1-m))*(m) - 4.2110604493513454e-05 * ([AD:Young]^2)*((1/m)^2)*(1/m) + 0.8242842812529125 * ([AD:Volume]^2)*((1-m))*(m) + 0.15740608991674646 * ([AD:Volume]^2)*((1-m)^2)*(m) - 0.02988849301491303
Lasso L0: 4 coef
Mean absolute error: 0.04821254480725535
Mean squared error: 0.005717845513924676
Mean absolute percentage error: 0.13154004054759158


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18163/603291486.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18163/603291486.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]


In [17]:
LassoL0Fit(X_reduced, Y.copy(), 5);

Lasso: selected coefficients are: [ 0.11165872 -0.14434662  1.92765721  0.27141306 -0.26102082  1.65053546]
Lasso: selected features are: ['([AD:Young])*((1-m))*(m)' '([AD:Young]^2)*((1/m)^2)*(1/m)'
 '([AD:Volume]^2)*((1-m))*(m)' '([AD:Volume]^2)*((1-m)^2)*(m)'
 '([AD:Volume]^2)*((1-m)^2)*(m^2)']
Constructed function is: 0.12036340434457132 * ([AD:Young])*((1-m))*(m) - 6.584391217603447e-05 * ([AD:Young]^2)*((1/m)^2)*(1/m) + 0.9109301437026236 * ([AD:Volume]^2)*((1-m))*(m) + 0.23704539117668263 * ([AD:Volume]^2)*((1-m)^2)*(m) - 0.5383684531121969 * ([AD:Volume]^2)*((1-m)^2)*(m^2) - 0.04133971282399962
Lasso L0: 5 coef
Mean absolute error: 0.04953687470127054
Mean squared error: 0.005304545897984246
Mean absolute percentage error: 0.16812228088990394


/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18163/603291486.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mean_std.append(coefficients[i] * mean_selected[i]/std_selected[i])
/var/folders/_0/q128s8pd1lb95k4pms0qlpw40000gn/T/ipykernel_18163/603291486.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coefficients[i] = coefficients[i] / std_selected[i]
